In [2]:
from pdf2image import convert_from_path

import cv2
import numpy as np
import os


In [3]:
os.path.exists('../../songs/drum_sheet_music/the spirit of radio.pdf')

True

In [4]:
# Step 1: Load the image and convert to grayscale
image = convert_from_path(
	'../../songs/drum_sheet_music/the spirit of radio.pdf',
	dpi=300,
	output_folder="C:/Users/marve/ryanroche/Capstone/auto-chart-engine/songs/drum_sheet_music")[0]
image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

# Check if the image was loaded successfully
if image is None:
	raise FileNotFoundError("The image file could not be loaded. Please check the file path.")

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [ ]:
# Step 2: Convert to a binary image using Otsu's thresholding
_, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
# Note: Using THRESH_BINARY_INV so that the symbols are white on a black background

In [ ]:
# Step 3: Remove staff lines using morphological operations
# Create a horizontal kernel (the length may need adjustment based on the sheet music)
horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1))
# Apply erosion followed by dilation (opening) to remove the horizontal lines
removed_lines = cv2.morphologyEx(binary, cv2.MORPH_OPEN, horizontal_kernel, iterations=1)

# Optionally subtract the detected staff lines from the binary image
cleaned = cv2.subtract(binary, removed_lines)

In [14]:
# Step 4: Display intermediate results for debugging
cv2.namedWindow("Cleaned (Symbols Only)", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Cleaned (Symbols Only)", 1200, 800)

# cv2.imshow('Original Grayscale', gray)
# cv2.imshow('Binary Image', binary)
# cv2.imshow('Removed Staff Lines', removed_lines)
cv2.imshow('Cleaned (Symbols Only)', cleaned)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# Step 5: Template matching example (for detecting a notehead)
# Load a template image of a notehead (make sure it is preprocessed similarly)
template = cv2.imread('notehead_template.jpg', 0)
w, h = template.shape[::-1]

# Perform template matching on the cleaned image
result = cv2.matchTemplate(cleaned, template, cv2.TM_CCOEFF_NORMED)
# Set a threshold for detection; adjust value as needed (for example, 0.7)
threshold = 0.7
locations = np.where(result >= threshold)

# Draw rectangles around detected noteheads
for pt in zip(*locations[::-1]):
    cv2.rectangle(image, pt, (pt[0] + w, pt[1] + h), (0, 0, 255), 2)

cv2.imshow('Detected Noteheads', image)
cv2.waitKey(0)
cv2.destroyAllWindows()